In [1]:
import pandas as pd
import numpy as np
import csv
from pathlib import Path

In [2]:
def read_and_process_and_export_human_AUT(input_file: str, output_file: str, dataset_directory: str, score: float=2., column: str='Responses') -> None:
    data = Path(dataset_directory) / input_file

    if not data.exists():
        print(f"cannot find {data}")

    df = pd.read_csv(data, sep='\t')
    df = df.fillna(0)
    # filter out lower-valued data - this has been evaluated by human scoreres
    df = df[df['average'] >= score]
    # we've got rid of some data, so let's reset the index so we have a sensible ID
    df = df.reset_index(drop=True)
    # export this bad boy

    output_dataset = Path(dataset_directory) / output_file
    
    df.to_csv(output_dataset, columns=[column], sep='\t')
    

In [3]:
dataset_directory = "/home/johnny/Documents/job-stuff/ElectricTwin/AUT-experiment1/LLM_AUT_test/human-responses"

In [4]:
input_fnam = "paperclip_AUT_dataset.csv"
output_fnam = "paperclip_human_AUT_dataset.tsv"

read_and_process_and_export_human_AUT(input_file = input_fnam, 
                                      output_file = output_fnam, 
                                      dataset_directory=dataset_directory)


In [5]:
input_fnam = "bowl_AUT_dataset.csv"
output_fnam = "bowl_human_AUT_dataset.tsv"

read_and_process_and_export_human_AUT(input_file = input_fnam, 
                                      output_file = output_fnam, 
                                      dataset_directory=dataset_directory)